<a href="https://colab.research.google.com/github/phisan-chula/2021-LDP_Design/blob/main/LDP_Mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Design of a Low Distortion Projection for a Mini Project**</br>  
LDP_Inspection : program to design  low distortion projections (LDPs) using conformal map projections for minimizing linear distortion between projected coordinates eg. UTM grid and the true distance at the surface of the engineering project.</br>  

Phisan Santitamonont,</br>  
Faculty of Engineering, Chulalongkorn University © 2022</br>
*Phisan.Chula@gmail.com*

*** 1) Installing RTSD/CMU Thai Geoid Model 2017 ***

In [1]:
############# faster refering TGM17 need Google Drive ####################
#from google.colab import drive
#drive.mount('/content/drive')
#TGM17 = r'/content/drive/MyDrive/Geodesy_Cache/tgm2017-1.pgm' 
import os
TGM17="/content/tgm2017-1.pgm"
os.environ['TGM17'] = TGM17 

In [2]:
%%capture
%%shell
echo $TGM17
if [ -f "$TGM17" ]; then
  echo "$TGM17 exists..."
else
  apt-get install subversion
  svn checkout  "https://github.com/phisan-chula/Construction_Survey/trunk/tgm2017-1"
  cat tgm2017-1/x* > "/content/tgm2017-1.pgm"
fi

In [3]:
%%capture
! pip install pygeodesy
! pip install pyproj
! pip install requests
! pip install folium

In [4]:
import requests
import pygeodesy as pgd
import numpy as np
from shapely.geometry import Point
import pandas as pd
import pyproj
###########################################################
#  Projection Plane (PP) @ [lat,lng] ,horizontal , vertical  buffer --> topo(SRTM/MSL) height
# designed central meridian degree, minute , false easting, false northing
POS, HOR_BUF,  VER_BUF =[13.5411528, 99.8240430], 2_000, 20.0 # Rachburi,
CM_DEG, CM_MIN, FALSE_E, FALSE_N = 99, 49, +2500, -1490000
###########################################
#POS, HOR_BUF,  VER_BUF =[14.519354, 101.017051], 1_000, 50  # CU-SBR
#CM_DEG, CM_MIN, FALSE_E, FALSE_N = 101, 60, +2500, -1490000
###########################################################
COL_LDP = ['UNDUL', 'h','HSF','PSF','CSF', 'CSF_ppm', 'LDP_E', 'LDP_N']
FLT_MAP =  { 'MSL': '{0:.1f}', 'UNDUL': '{0:.1f}', 'h': '{0:.1f}', 'CSF_ppm': '{0:.1f}', 'LDP_E': '{0:,.3f}',  'LDP_N': '{0:,.3f}' }
TM = r'+proj=tmerc +lat_0=0.0 +lon_0={} +k_0={}  +x_0={}  +y_0={}  +a={} +b={} +units=m +no_defs'
###########################################################
res = requests.get( r'https://api.opentopodata.org/v1/srtm30m?locations={},{}'.format( *POS ) )
MSL = res.json()['results'][0]['elevation']
ELLPS  = pgd.datums.Ellipsoids.WGS84
GEOID = pgd.geoids.GeoidKarney( TGM17 )

In [5]:
def dd2DMS( dd, PREC=7, POS=''  ):
    '''convert degree to DMS string'''
    return pgd.dms.toDMS( dd, prec=PREC,pos=POS )
    
def CalcLDP( row ):
    UNDUL = GEOID.height( row.lat,row.lng )
    RG   = ELLPS.rocGauss( row.lat )
    h     = UNDUL + row.MSL
    HSF = RG/(RG+h)
    PSF = pyproj.Proj( LDP).get_factors( row.lng, row.lat ).meridional_scale
    CSF = PSF*HSF
    CSF_ppm = (CSF-1)*1E6
    TR = pyproj.Transformer.from_crs( 'epsg:4326', LDP )
    LDP_E,LDP_N = TR.transform( row.lat, row.lng )
    return [UNDUL, h, HSF, PSF, CSF, CSF_ppm, LDP_E, LDP_N]

In [13]:
res = requests.get( r'https://api.opentopodata.org/v1/srtm30m?locations={},{}'.format( *POS ) )
MSL = res.json()['results'][0]['elevation']
####################################################
UNDUL = GEOID.height( *POS )
HAE = UNDUL + MSL                 # h = N + H
RG = ELLPS.rocGauss( POS[0] )  # RG = sqrt(MN)
k0 = np.round(1 + HAE/RG, 6)    #  M.Dennis 2016 : Ground Truth ... (...5 to 6 digits)
print( f'Projection Plane    : lat = {dd2DMS(POS[0]):}  lng = {dd2DMS(POS[1]):}  ==> k0 = {k0:.6f} ')
print( f'Topography          :  MSL = {MSL:.1f} m. HAE={HAE:.1f} m. ,  N={UNDUL:.1f} m.  ' )
print( f'''Designed LDP-TM :  {CM_DEG:.0f}\u00B0 {CM_MIN:.0f}\u00B4'''\
          f'''    FALSE_E={FALSE_E:+,.0f}m.   FALSE_N={FALSE_N:+,.0f}m ''' )

Projection Plane    : lat = 13°32′28.15008″  lng = 99°49′26.5548″  ==> k0 = 0.999996 
Topography          :  MSL = 5.0 m. HAE=-26.9 m. ,  N=-31.9 m.  
Designed LDP-TM :  99° 49´    FALSE_E=+2,500m.   FALSE_N=-1,490,000m 


In [7]:
LDP = pyproj.CRS( TM.format( CM_DEG+CM_MIN/60., k0, FALSE_E, FALSE_N, ELLPS.a, ELLPS.b ) )
# create 3 projection planes and test them
EWNS =Point( POS[1],POS[0] ).buffer( HOR_BUF/111_000 , cap_style = 3  ).exterior.coords.xy
PP_EWNS = np.vstack( (np.array( [POS[1],POS[0]] ) , np.array(EWNS).T) )[:-1]
dfPP = pd.DataFrame( {'Point':['P0','P1','P2','P3','P4'], 'lng':PP_EWNS[:,0], 'lat':PP_EWNS[:,1]  } )
dfPP = pd.concat( 3*[dfPP] ,  ignore_index=True) # create 3 planes ...
dfPP['MSL'] = 5*[MSL+VER_BUF]+  5*[MSL] +  5*[MSL-VER_BUF]
dfPP[COL_LDP] = dfPP.apply( CalcLDP, axis=1, result_type='expand')
print(   '\033[91m' , LDP )
dfPP.style.format(FLT_MAP)

 +proj=tmerc +lat_0=0.0 +lon_0=99.81666666666666 +k_0=0.999996  +x_0=2500  +y_0=-1490000  +a=6378137.0 +b=6356752.314245179 +units=m +no_defs +type=crs


,Point,lng,lat,MSL,UNDUL,h,HSF,PSF,CSF,CSF_ppm,LDP_E,LDP_N
0,P0,99.824043,13.541153,25.0,-31.9,-6.9,1.000001,0.999996,0.999997,-2.9,"3,298.447","7,574.087"
1,P1,99.842061,13.559171,25.0,-31.9,-6.9,1.000001,0.999996,0.999997,-2.8,"5,248.591","9,567.637"
2,P2,99.842061,13.523135,25.0,-31.8,-6.8,1.000001,0.999996,0.999997,-2.8,"5,249.005","5,580.800"
3,P3,99.806025,13.523135,25.0,-31.9,-6.9,1.000001,0.999996,0.999997,-2.9,"1,348.010","5,580.682"
4,P4,99.806025,13.559171,25.0,-32.0,-7.0,1.000001,0.999996,0.999997,-2.9,"1,348.183","9,567.520"
5,P0,99.824043,13.541153,5.0,-31.9,-26.9,1.000004,0.999996,1.000000,0.2,"3,298.447","7,574.087"
6,P1,99.842061,13.559171,5.0,-31.9,-26.9,1.000004,0.999996,1.000000,0.3,"5,248.591","9,567.637"
7,P2,99.842061,13.523135,5.0,-31.8,-26.8,1.000004,0.999996,1.000000,0.3,"5,249.005","5,580.800"
8,P3,99.806025,13.523135,5.0,-31.9,-26.9,1.000004,0.999996,1.000000,0.3,"1,348.010","5,580.682"
9,P4,99.806025,13.559171,5.0,-32.0,-27.0,1.000004,0.999996,1.000000,0.3,"1,348.183","9,567.520"


**User will input project's control points, RTKs, setting-out coordinates here...**

In [8]:
from io import StringIO
CSV = '''Point,lat,lng,MSL
RTK1,13.542153,99.824043,20.0
RTK2,13.540153,99.821043,-30.0
RTK3,13.541153,99.822043,2.0
'''
CALC_RTK = True
if CALC_RTK:
    dfRTK = pd.read_csv( StringIO(CSV) )
    dfRTK[COL_LDP] = dfRTK.apply( CalcLDP, axis=1, result_type='expand')
dfRTK.style.format(FLT_MAP)
    

,Point,lat,lng,MSL,UNDUL,h,HSF,PSF,CSF,CSF_ppm,LDP_E,LDP_N
0,RTK1,13.542153,99.824043,20.0,-31.9,-11.9,1.000002,0.999996,0.999998,-2.1,"3,298.444","7,684.743"
1,RTK2,13.540153,99.821043,-30.0,-31.9,-61.9,1.000010,0.999996,1.000006,5.7,"2,973.716","7,463.466"
2,RTK3,13.541153,99.822043,2.0,-31.9,-29.9,1.000005,0.999996,1.000001,0.7,"3,081.958","7,574.103"


In [9]:
import folium
from folium.features import DivIcon

icon=folium.Icon(color='red', icon='plus' )
map = folium.Map(location =[POS[0],POS[1]],  zoom_start = 14 )
for grp,row in dfPP.groupby( [ 'Point' ] ):
    csf = list( row.CSF_ppm.round(1) ); pnt = '{:}'.format( row.iloc[0]['Point'] )
    location = (row.iloc[0].lat , row.iloc[0].lng)
    folium.CircleMarker(location=location, radius=15, color='red', fill_color ='red', fill_opacity=0.5,
                                tooltip=f'<b>CSF:{csf:}</b>').add_to(map)
    folium.map.Marker(location, icon=DivIcon( icon_size=(30,30), icon_anchor=(5,14),
                          html=f'<div style="font-size: 14pt">{pnt:}</div>' ) ).add_to(map)
if CALC_RTK:
    for i,row in dfRTK.iterrows():
        location = (row.lat,row.lng) ; pnt = '{}:[{}m]'.format( row['Point'], row['MSL'] )
        folium.CircleMarker(location=location, tooltip=f'<b>CSF:{row.CSF_ppm:}</b>',
                            radius=15, color='green', fill_color ='green', fill_opacity=0.5 ).add_to(map)
        folium.map.Marker(location, icon=DivIcon( icon_size=(30,30), icon_anchor=(5,14),
                       html=f'<div class="center" style="font-size: 16p"><p>{pnt:}</p></div>'  ) ).add_to(map)
map